# Script for predictions 

Here, we assume that there is a saved model which we want to use for predictions, and use it on a target image.

In [48]:
# Imports
import os, re, pathlib, time, datetime, argparse, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from src import helper_models, helper_data
from PIL import Image

In [198]:
# Paths
modelpath='./out/conv2/2020-01-19_22h44m18s/'
argsname=modelpath+'args.txt'
im_name='./images/SPC-EAWAG-0P5X-1528200107985735-536606102621-000979-002-928-2332-60-48.jpeg'

In [200]:
# Read Arguments
print(args)
layers=[None,None]
for s in re.split('[\,,\),\(]',args):
    if 'height' in s:
        height=np.int64(re.search('(\d+)',s).group(1))
    if 'width' in s:
        width=np.int64(re.search('(\d+)',s).group(1))
    if 'depth' in s:
        depth=np.int64(re.search('(\d+)',s).group(1))
    if 'model' in s:
        model=re.search('=\'(.+)\'$',s).group(1)
    if 'resize' in s:
        resize=re.search('=\'(.+)\'$',s).group(1)
    if 'layers' in s: #first layer
        layers[0]=np.int64(re.search('=\[(.+)$',s).group(1))
    if re.match('^ \d+',s): #second layer
        layers[1]=np.int64(re.match('^ (\d+)',s).group(1))
    if 'datapath' in s:
        datapath=re.search('=\'(.+)\'$',s).group(1)
    if 'outpath' in s:
        outpath=re.search('=\'(.+)\'$',s).group(1)
    
classes = [ name for name in os.listdir(datapath) if os.path.isdir(os.path.join(datapath, name)) ]


Namespace(aug=True, bs=32, datapath='./data/zooplankton_trainingset_15oct/', depth=3, height=128, layers=[256, 128], lr=0.0001, model='conv2', opt='sgd', outpath='./out/', plot=False, resize='keep_proportions', testSplit=0.2, totEpochs=5000, verbose=False, width=128)



In [193]:
from stat import S_ISREG, ST_CTIME, ST_MODE
# # Initialize and load model
# if model == 'mlp':
#     model = helper_models.MultiLayerPerceptron.build(input_size=len(data[0]), classes=len(classes), layers=layers)
# elif model == 'conv2':
#     model = helper_models.Conv2Layer.build(width=width, height=height, depth=depth, classes=len(classes))
# elif model == 'smallvgg':
#     model = helper_models.SmallVGGNet.build(width=width, height=height, depth=depth, classes=len(classes) )
# else:
#     raise NotImplementedError('Not implemented model {}'.format(model))


# Choose backup
backupdir = outpath+'/'+model+'/'+os.listdir(outpath+'/'+model)[0] # Get the first one. We have no criterion currently.
entries = [backupdir+'/'+entry for entry in os.listdir(backupdir)]
entries = ((os.stat(path), path) for path in entries)
entries = ((stat[ST_CTIME], path) for stat, path in entries if S_ISREG(stat[ST_MODE]))
backupfile=sorted(entries)[-1][1]

model = keras.models.load_model(backupfile)


In [201]:
# Load image
image = Image.open(im_name)
if resize == 'acazzo':
    image = image.resize((width,height))
else:
    # Set image's largest dimension to target size, and fill the rest with black pixels
    image,rescaled = helper_data.ResizeWithProportions(image, width) # width and height are assumed to be the same (assertion at the beginning)
npimage=np.array(image.copy())
if model == 'mlp':
    npimage = npimage.flatten()
image.show()
image.close()
    


In [ ]:
# Print prediction
print('Class: ',model.predict(image))